In [ ]:
import requests
import RPi.GPIO as GPIO

# LED 핀 번호 설정 (BCM 모드)
LED1_PIN = 18
LED2_PIN = 23

def setup_leds():
    GPIO.setmode(GPIO.BCM)
    GPIO.setup(LED1_PIN, GPIO.OUT)
    GPIO.setup(LED2_PIN, GPIO.OUT)


def get_weather_by_city_id(city_id, api_key):
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = f"{base_url}id={city_id}&appid={api_key}&units=metric"

    response = requests.get(complete_url)
    data = response.json()

    if data["cod"] != "404":
        city_name = data["name"]
        main_data = data["main"]
        weather_data = data["weather"][0]
        temp = main_data["temp"]
        humidity = main_data["humidity"]
        description = weather_data["description"]

        return f"{city_name}의 현재 날씨는 {description}, 온도는 {temp}℃이며 습도는 {humidity}%입니다."
    else:
        return "날씨 정보를 가져오는 데 실패했습니다."

def main():
    
    setup_leds()
    
    city_ids = {
        1: 1840898, 2: 1835847, 3: 1843561, 4: 1835553, 5: 1845759, 
        6: 1835224, 7: 1845457, 8: 1841808, 9: 1844751, 10: 1846265, 
        11: 1845136, 12: 1836553, 13: 1843137, 14: 1846986, 15: 1835327, 
        16: 1839071, 17: 1833742, 18: 1838519
    }
    
    api_key = "5c259fe39c653fa07d0efaa6c8455d1f"  # OpenWeatherMap에서 발급받은 API 키로 변경하세요.

    while True:
        try:
            user_choice = int(input("도시 번호를 입력하세요 (1~18): "))
            if user_choice in city_ids:
                city_id = city_ids[user_choice]
                result = get_weather_by_city_id(city_id, api_key)
                print(result)
                
                # 날씨 정보에 따라 LED 제어
                if 'clear' in result.lower():
                    GPIO.output(LED1_PIN, GPIO.HIGH)
                    GPIO.output(LED2_PIN, GPIO.LOW)
                elif 'rain' in result.lower():
                    GPIO.output(LED1_PIN, GPIO.LOW)
                    GPIO.output(LED2_PIN, GPIO.HIGH)
                else:
                    GPIO.output(LED1_PIN, GPIO.LOW)
                    GPIO.output(LED2_PIN, GPIO.LOW)

                
            else:
                print("유효한 도시 번호를 입력하세요.")
        except ValueError:
            print("올바른 숫자를 입력하세요.")

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        # Ctrl+C를 누르면 GPIO 설정 초기화 후 종료
        GPIO.cleanup()

